In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
torch._dynamo.config.disable = True
torch.cuda.is_available()

True

In [4]:
# this one fix charmap issue in windows, but in linux as in collab no problem
import builtins
original_open = builtins.open
def utf8_open(*args, **kwargs):
    if 'mode' in kwargs and 'b' in kwargs['mode']:
        return original_open(*args, **kwargs)
    if len(args) >= 2 and 'b' in args[1]:
        return original_open(*args, **kwargs)
    if 'encoding' not in kwargs and len(args) < 3:
        kwargs['encoding'] = 'utf-8'
    return original_open(*args, **kwargs)
builtins.open = utf8_open

In [4]:
from unsloth import FastLanguageModel
max_seq_length = 2048
dtype = None
load_in_4bit = True

# fourbit_models = [
#     "unsloth/Llama-3.2-1B-bnb-4bit",
# ]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


d:\brofinetune\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


d:\brofinetune\.venv\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:339: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"{DEVICE_TYPE}:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.8.1: Fast Llama patching. Transformers: 4.55.0.
   \\   /|    NVIDIA GeForce RTX 4080 SUPER. Num GPUs = 1. Max memory: 15.992 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.7.1+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.8.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
from datasets import Dataset

In [6]:
from datasets import Dataset
import json
from unsloth.chat_templates import get_chat_template

# Load local training data
with open('training_data.json', 'r') as f:
    data = json.load(f)

# Convert to conversation format
conversations = []
for i in range(0, len(data), 2):
    if i + 1 < len(data):
        conversation = [
            {"role": data[i]["role"], "content": data[i]["content"]},
            {"role": data[i+1]["role"], "content": data[i+1]["content"]}
        ]
        conversations.append({"conversations": conversation})

# Create dataset
dataset = Dataset.from_list(conversations)

# Setup chat template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

# Format conversations for training
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

# Apply formatting
dataset = dataset.map(formatting_prompts_func, batched = True)
print(f"Dataset size: {len(dataset)}")
print(f"Sample: {dataset[0]['text'][:200]}...")

Map: 100%|██████████| 57/57 [00:00<00:00, 3453.55 examples/s]

Dataset size: 57
Sample: <|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the capital of ...


In [8]:
dataset[0]['conversations']

[{'content': 'What is the capital of France?', 'role': 'user'},
 {'content': 'Yo bro! The capital of France is Paris, dude. Beautiful city with the Eiffel Tower and all that good stuff. You thinking of visiting or just curious? Either way, you got this!',
  'role': 'assistant'}]

In [9]:
dataset[0]['text']

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat is the capital of France?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nYo bro! The capital of France is Paris, dude. Beautiful city with the Eiffel Tower and all that good stuff. You thinking of visiting or just curious? Either way, you got this!<|eot_id|>'

In [10]:
from trl import SFTConfig, SFTTrainer
from transformers import DataCollatorForSeq2Seq
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    packing = False,
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1,
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
        dataset_num_proc = 1,  # This line solve issue in windows, linux is fine
    ),
)


Unsloth: Tokenizing ["text"]: 100%|██████████| 57/57 [00:00<00:00, 5990.36 examples/s]


In [11]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=32): 100%|██████████| 57/57 [00:12<00:00,  4.61 examples/s]


In [12]:
print(tokenizer.decode(trainer.train_dataset[5]["input_ids"]))

<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

I failed my driving test.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Aw man, that's a bummer, but hey - tons of people don't nail it on the first try! It's totally normal, bro. Just means you get another shot to show 'em what you're made of. Take a few more practice runs, maybe work on whatever tripped you up, and you'll get it next time for sure. Don't let it get you down - you're gonna be cruising around soon enough!<|eot_id|>


In [13]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]]).strip()

"Aw man, that's a bummer, but hey - tons of people don't nail it on the first try! It's totally normal, bro. Just means you get another shot to show 'em what you're made of. Take a few more practice runs, maybe work on whatever tripped you up, and you'll get it next time for sure. Don't let it get you down - you're gonna be cruising around soon enough!<|eot_id|>"

In [14]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4080 SUPER. Max memory = 15.992 GB.
3.168 GB of memory reserved.


In [15]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 57 | Num Epochs = 8 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)


Step,Training Loss
1,2.362000
2,2.425000
3,2.366000
4,2.340100
5,2.507300
6,2.085300
7,2.036700
8,2.284500
9,1.782800
10,1.763200


In [16]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

47.1693 seconds used for training.
0.79 minutes used for training.
Peak reserved memory = 3.328 GB.
Peak reserved memory for training = 0.16 GB.
Peak reserved memory % of max memory = 20.81 %.
Peak reserved memory for training % of max memory = 1.001 %.


In [21]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "I'm so sad. My interview was really off."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
response = tokenizer.batch_decode(outputs)
for res in response:
    print(res)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm so sad. My interview was really off.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Aw man, interviews are always nerve-wracking, but that's totally normal! Most interviews are way more relaxed than actually being in that interview, bro. This probably just means you're meant for another one - maybe an even better one! Don't let one tiny moment bring you down, dude - you got this


In [22]:
input_length = inputs.shape[1]
response = tokenizer.decode(outputs[0][input_length:], skip_special_tokens=True)
print(response)

Aw man, interviews are always nerve-wracking, but that's totally normal! Most interviews are way more relaxed than actually being in that interview, bro. This probably just means you're meant for another one - maybe an even better one! Don't let one tiny moment bring you down, dude - you got this


In [20]:
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
_ = model.generate(input_ids=inputs, streamer=text_streamer, max_new_tokens=128,
                   use_cache=True, temperature=1.5, min_p=0.1)

Ha man, that's a trick question! Paris isn't a city-state, it's a part of France. The capital of France is Paris itself, bro! You're thinking of the 'Eiffel Tower capitol' like in that movie - just Paris though, dude!


In [23]:
# Push to HuggingFace Hub (optional)

import os
from dotenv import load_dotenv

load_dotenv()

hg_token = os.getenv('HG_TOKEN')
model.push_to_hub("doublebank/bro-chatbot", token=hg_token)
tokenizer.push_to_hub("doublebank/bro-chatbot", token=hg_token)

# Load from Hub later
# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "doublebank/bro-chatbot",
#     max_seq_length = 2048,
#     dtype = None,
#     load_in_4bit = True,
# )


100%|██████████| 1/1 [00:04<00:00,  4.24s/it]


Saved model to https://huggingface.co/doublebank/bro-chatbot


100%|██████████| 1/1 [00:03<00:00,  3.98s/it]


In [26]:
from huggingface_hub import upload_file

# Then upload additional files
upload_file(
    path_or_fileobj="MODEL_CARD.md",
    path_in_repo="README.md",
    repo_id="doublebank/bro-chatbot",
    token=hg_token
)

upload_file(
    path_or_fileobj="training_data.json",
    path_in_repo="training_data.json", 
    repo_id="doublebank/bro-chatbot",
    token=hg_token
)

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/doublebank/bro-chatbot/commit/bb6013fee2590d8b17ea99a9cede3606e2f2c300', commit_message='Upload training_data.json with huggingface_hub', commit_description='', oid='bb6013fee2590d8b17ea99a9cede3606e2f2c300', pr_url=None, repo_url=RepoUrl('https://huggingface.co/doublebank/bro-chatbot', endpoint='https://huggingface.co', repo_type='model', repo_id='doublebank/bro-chatbot'), pr_revision=None, pr_num=None)

In [7]:
from unsloth import FastLanguageModel

# Load from the outputs directory (latest checkpoint)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "./outputs/checkpoint-60",  # or "./outputs/checkpoint-60" if specific
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

# # Setup for inference
# from unsloth.chat_templates import get_chat_template
# tokenizer = get_chat_template(tokenizer, chat_template = "llama-3.1")
# FastLanguageModel.for_inference(model)

# # Test it
# from hg_chat import HGChat
# bro_chat = HGChat("./outputs")  # Load from local checkpoint
# response = bro_chat.chat("Hey bro, how are you?")
# print(response)


==((====))==  Unsloth 2025.8.1: Fast Llama patching. Transformers: 4.55.0.
   \\   /|    NVIDIA GeForce RTX 4080 SUPER. Num GPUs = 1. Max memory: 15.992 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.7.1+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.8.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
# Save the recovered model
model.save_pretrained("./bro-chatbot-final")
tokenizer.save_pretrained("./bro-chatbot-final")

# Push to HuggingFace
model.push_to_hub("your-username/bro-chatbot", token="your_token")
tokenizer.push_to_hub("your-username/bro-chatbot", token="your_token")
